# Part 2: Too Fitted [WIP]

In the last section, we learned how to facilitate the building of iris classification rules by using an out-of-the-box statistical tool called principal component analysis (PCA) to build a compact representation of the input data. Using PCA, we built a simple, linear rule that allowed us to classify our data with over 96% accuracy!

However, as we hinted at the bottom of that section, this 96% accuracy figure is misleading. The data we used to score ourselves was the same data that we used to build our rule, so it seems natural to assume the rule itself would work unusually well on that data (we're not completely stupid, after all). But the reality is that I don't _care_ about that data: I already know what irises they are! What I want to know is how well my rule is going to work on new data, samples for which I don't know the answer and am therefore relying on my rule to provide one.

Of course, it might be reasonable to assume that my performance on the data that informed the rule, called the **training set**, will provide at least a decent approximation of my "true" performance on the problem (for a certain sense of the word "true" that we'll talk about momentarily). But for obvious reasons, it seems likely to be an overestimate. Estimating how _much_ of an overestimate it is accounts for much of the art of machine learning, and it is that precise problem that we'll be discussing in this section.

## Functions of a Different Color
Let's start by framing the problem once again, focusing on our iris problem for the purposes of concreteness. We have some measurements about iris petals and sepals, and for any given set of measurements would like to assign a label describing what type of iris it is. Mathematically, we're looking for a **function** $f$ that maps from our input measurements, which we'll describe with a **vector** $\vec{x}$, to an output **label** $y$:
$$y = f(\vec{x})\ .$$

Now, presumably there is some _true_ function nature uses to do this mapping: after all, these measurements and labels came from _somewhere_. The problem is that we don't know what that function is (if we did, we'd be physicists). So the program of machine learning is to hypothesize a _family_ of functions $\mathcal{F}$, in the hope that there is a member of that family that "looks" like this true function, enough to mimic its outputs under most circumstances. Machine learning algorithms use our data, which amount to glimpses at what this function does, to find that member of our hypothesized family which best interpolates the true function on those glimpses.

So in this paradigm, there are really three functions of interest:
- The true function $f$, which nature used to produce our data in the first place.
- The member $f^*$ of our family of functions $\mathcal{F}$ that _looks_ the most like $f$ (for a loose sense of "looks like" that we'll formalize later).
- The member $\hat{f}$ of $\mathcal{F}$ that performs best on our data.

Building a good machine learning model, then, relies on two strokes of luck.
1. Picking a family of functions $\mathcal{F}$ that has at least _one_ member that approximates $f$ well (that is, $f^* \approx f$ for most data of interest).
2. Finding that best member (or at least one close to it) among the many possible choices in $\mathcal{F}$.

Most of the subtle and challenging aspects of machine learning boil down to the fact that for most problems and families of functions $\mathcal{F}$, $\hat{f}$ and $f^*$ are _not_ the same function. As such, our job as data scientists is largely to manipulate our data, constrain the family $\mathcal{F}$, and construct clever algorithms in such a way as to bring $\hat{f}$ and $f^*$ into closer alignment. This way, building a model which performs maximally well on the data correlates better with building a model which _actually_ solves the problem at hand well.

## My Extremely Artificial Example That You Should Not Try to Over-Generalize From but Instead Use to Form Intuitions That You Should Expect to be Challenged Constantly
But this is all getting pretty theoretical, let's come back down to reality with a concrete and _extremely_ artificial example that can help us build some intuition on these concepts. Once we've beaten that example to death, we can return to our iris example, which, though simple by comparison to most ML problems, will still be orders of magnitude more complex than the one I'm about to present.

Pretend you're a freelance data scientist, and I come to you with the following input and output observations from some problem I'm interested in (but because it's highly sensitive, I can't tell you what that problem is or what these measurements represent).

In [1]:
from secret_problem import data
import plot_utils
from bokeh.io import output_notebook, show
output_notebook()

N = len(data)
p = plot_utils.make_2d_scatter_plot(
    data,
    fig_kwargs={
        "title": f"{N} Observations from Top Secret Problem",
        "x_axis_label": "Input Measurement",
        "y_axis_label": "Output Measurement",
    }
)
show(p)

Loading BokehJS ...

My ask of you, talented data scientist that you are, is to build me a function which can predict what the corresponding output measurement will be if presented with a _new_ input measurement. Since you have no idea what these measurements represent, and so have no intuition about what the relationship between these two variables should be, you have no choice but to make a guess at this function using only the data I've given you.

So how to proceed? Well, what sort of relationships can we observe about the inputs and outputs from the data? There certainly seems to be _some_ sort of correlation between them (i.e. larger input values tend to correspond to larger output values), so why don't we start by adopting the simplest model we can: let's assume that this data falls roughly along a straight line, then try to find the straight line that best captures these observations.

Of course, this data doesn't actually fall along a straight line, and so our guesses are almost certain to be _wrong_. But if we accept that being 100% right all of the time is not a realistic (or, for a lot of reasons, desirable) goal, then we can focus on the much more tractable task of being as not-wrong as possible _on average_. So let's start with this simple model, evaluate our level of wrongness, then decide how adding more complexity might move this level up or down (on average).

More specifically, we'll assume that the relationship of the output variable $y$ and the input variable $x$ is of the form

$$y = mx + b + \text{noise}$$

for some values $m$ and $b$ $\text{noise}$ represents some unavoidable jitter in our observations (and represents some minimum level of wrongness we can't expect to improve upon). $m$ and $b$ represent the parameters of our family of functions $\mathcal{F}$, the family of all straight lines.

By making this assumption and adopting this model (restricting our search to this family of functions), we've reduced our problem to finding an $m$ and $b$ that minimize some measure of "error" over our observations. For a **regression** problem like this, it's common to use the square of the difference between the guess our model produces $\hat{y}$ and the measured value $y$:

$$\text{Error}(y, \hat{y})\ =\ (y - \hat{y})^2$$

So if we have $N$ observations, we're looking for an $m$ and a $b$ that minimize

$$\sum_{i=1}^N (y_i - \hat{y}_i)^2\ =\ \sum_{i=1}^N (y_i - mx_i - b)^2$$

for each of our observation pairs $(x_i, y_i)$. This formulation is great, because it makes our task extremely concrete and comparatively simple!

How do we find that $m$ and $b$? One method might be to proceed by brute force: check a lot of different combinations, evaluate the amount of error they incur on our dataset, then pick the combination that performs the best. The drawback to this is that I may need to check a _lot_ of values, and even then I have no way of knowing if a better guess might be waiting out there that I didn't try.

Luckily, we can use some pretty simple calculus to arrive at an **analytical** solution for the best $m$ and $b$: a neat formula that we can use our computer to execute for us. In fact, this is one of the main reasons I picked this model in the first place: it can be solved simply and exactly! Of course, the price for this simplicity is a model that doesn't quite fit right; we've already noted that my data quite obviously doesn't follow a straight line. But a simple model that I can solve is worth much more than a complex model whose solution I can only guess at (insert lengthy list of caveats here).

I won't cover the derivation of the solution, but the answer comes out to (adding in hats to represent that these are our estimates based on our data, and disregarding the neater formulation we could come up with if we formulated things in terms of averages)

$$\hat{m}\ =\ \frac{\frac{1}{N}\sum{y_i} \sum{x_i} - \sum{y_i x_i}}{\frac{1}{N}(\sum{x_i})^2 - \sum{x_i^2}}$$
$$\hat{b}\ =\ \frac{1}{N}\sum{y_i} - \frac{m}{N} \sum{x_i}$$

Let's calculate these values for our dataset and plot the corresponding line over our data to see how close it fits.

In [2]:

m = (
    (data.y.sum()*data.x.sum() / N - (data.y*data.x).sum()) /
    ((data.x.sum())**2 / N - (data.x**2).sum())
)
b = data.y.sum()  / N - m * data.x.sum() / N

p.line(
    [data.x.min(), data.x.max()],
    [m*data.x.min() + b, m*data.x.max() + b],
    line_color="#222222",
    line_dash="2 2",
    legend_label="Best fit"
)
show(p)

So as we expected, things aren't perfect, but it looks like our line runs more or less down the center of our data, which is about all we can ask for. For the sake of concreteness, let's evaulate our error over the dataset to put a number on how wrong our function is. (I'll also normalize by the number of obseravtions for the sake of consistency with what we'll do later. This doesn't affect the fit parameters.)

In [3]:
y_hat = m * data.x + b
error = ((data.y - y_hat)**2).mean()
print(f"Squared error: {error:0.2f}")

Squared error: 23.45


This number in and of itself doesn't mean much, especially since we don't know what these numbers represent and so have no sense for how well we "should" be able to do. But it gives us a baseline against which to compare other, possibly more complex models, and helps us find out if our changes solve the problem better or worse.

However, there's already a lot of problems we need to address with this one simple model. For starters, we've once again used the same data we're using to evaluate ourselves that we used to build our model in the first place. To see why this is such a problem, take for example input values around 1.4. we see that we have output values coming in as low as 2 and as high as 16, with a range of other values in between. What if it turns out that numbers closer to 16 are actually more likely, and we just had bad luck by collecting some improbably low output measurements when we built our dataset? Our model would then perform _worse_ than this measured value (on average) on new observations. If our application relied on some promise of model quality, things could go very wrong very quickly if our estimate of model quality is far enough off.

If we could observe, say, a million examples of input/output pairs with input around 1.4, we could gain a better sense for what to expect here, but we're stuck with the data we have. How can we better estimate how our models will perform "in the wild"?

### Ready to see the Oracle - What's really happening
But there's an even bigger issue to address: it turns out that the function nature (a.k.a. my code in `secret_problem.py`) used to generate this data wasn't a line at all! The *true* function $f$ that relates these observations is

$$y\ =\ 2x^2 + 4x + 1 + \mathcal{N}(0, 16)$$

where $\mathcal{N}(0, 16)$ represents a random variable drawn from a normal distribution with mean 0 and variance 16. We've picked the wrong family of functions altogether! Obviously, we did it for good reasons: we didn't know this fact in advance, and so a line was about as good a guess as we could muster and was easily solvable, so why not use it?

That said, we're clearly doomed to incur some minimum level of error from the sheer fact that we've picked a "bad" model: even the best line doesn't look like a quadratic _all_ the time. Different values of $m$ and $b$ will approximate this quadratic better or worse (as measured by their average error across _all_ possible samples, weighted by the likelihood of those samples), and all we could do is find the line which approximates it best.

As it turns out, we can actually come up with a formula for the average error incurred by a given $(m, b)$ pair on the _real problem_, that is, using the _actual_ distributions of $x$ and $y$ that we, in general, dont' know (but do in this case thanks to the artificiality of the problem). For a generic (noisy) quadratic of the form $y = \alpha x^2 + \beta x + \gamma + \mathcal{N}(0, \sigma^2)$, modelling it with a line produces an average error of:

$$J(m, b)\ =\ 2\alpha^2 + (m - \beta)^2 + (b - \alpha - \gamma)^2 + \sigma^2$$

This formulation of error as a function of two variables, the slope and intercept of the line used to fit the dataset, invites us to draw a **cost surface**, plotting this cost as a heat map across different values of $m$ and $b$. The _true_ best values of $m$ and $b$ (corresponding to $f^*$ for the family $\mathcal{F}$ of straight lines for this problem) will be where this surface is "coldest", at the value

$$m\ =\ \beta$$
$$b\ =\ \alpha + \gamma$$

(I won't show the math to derive this either, but you can get it pretty quickly with some simple calculus on $J(m, b)$).

In [4]:
from secret_problem import plot_cost_surface

cost_surface = plot_cost_surface()
show(cost_surface)

But when we were fitting our model, we didn't get to know about this true cost surface. All we had to work with was the cost surface of how well each $(m, b)$ pair performed on our observations. What does _this_ surface look like?

In [5]:
empirical_cost_surface = plot_cost_surface(sample_size=250)
show(empirical_cost_surface)

So on this finite sample, our empirical cost surface looks quite different from the true cost surface, and so we end up picking parameters, corresponding to our guess $\hat{f}$, that perform _worse_ than the true best ones, the parameters of $f^*$ (the best line we could pick of all lines).

And this problem just gets worse the less data we have! Let's plot these two surfaces side by side, and add a slider that controls for the number of samples we use to calculate the empirical surface.

In [6]:
from secret_problem import plot_surfaces_side_by_side

side_by_side = plot_surfaces_side_by_side(
    fig_kwargs={"height": 300, "width": 300}
)
show(side_by_side)

At low numbers of observations (less than 20 or so), our empirical cost surface doesn't look anything like the true cost surface at all! As such, not only is our best fit model way off, but so is our estimate of that model's true performance. If you set the slider to 20, for example, you'll see that our empirical estimate of our model's error is around 14. But the true error, which we would measure as we used this model on many new samples and which you can see in the left surface, is really around 25!

The key take away is that we're running into two fundamental issues:
1. The best member $f^*$ of our proposed family of functions $\mathcal{F}$, the family of straight lines, doesn't look like the true function $f$, a quadratic. That's why we have a nontrivial amount of error even at the minimum of the cost surface (around 24 or so).
2. The cost surface that our best-fit formula "sees" is a rough approximation to the _true_ cost surface, and so we end up making bad guesses at $\hat{f}$ from our incomplete information that aren't sufficiently close to $f^*$. The more data we have, the more we can bring these surfaces into alignment, but we don't know in advance how much data we'll _need_, nor is it always trivial to get _more_ data. (There's also the issue that something about the way we sample data **biases** our cost surface towards wrong answers, but that's a much different, though equally important, discussion).

If we want to build good machine learning models, we need to get good at proposing families of models that have at least some "good" members among them, and at aligning the cost surface of our fitting procedure to the real one so that we can arrive at these good members. The great irony of ML, however, is that it turns out these tasks are largely anti-correlated: the more robust and expressive our proposed family of functions is, the harder it is in general to find the good members given the same amount of data. We'll explore this more in the following section.